# Analiza dnevnih migracij v Sloveniji

V nalogi bom analiziral migracije med Slovenskimi občinami in ugotavljal v kakšnem razmerju so cene nepremičnin ter  delovne razmere in ugotavljal kje bi bilo mogoče izboljšat zaslužek in standard življenja.

Na začetku naložimo vse podatke, ki jih bomo pri analizi uporabili.

In [1]:
import pandas as pd

# Nastavitve prikaza
pd.options.display.max_rows = 10

%matplotlib inline

# Podatki
import os.path

pot_do_podatkov = "./podatki"

nepremicnine = pd.read_csv(os.path.join(pot_do_podatkov, "nepremicnine.csv"), index_col="id")
podjetja = pd.read_csv(os.path.join(pot_do_podatkov, "podjetja.csv"))
razdalje_med_obcinami = pd.read_csv(os.path.join(pot_do_podatkov, "razdalje.csv"))
migracije_med_obcinami = pd.read_csv(os.path.join(pot_do_podatkov, "migracije.csv"))

obcine = pd.read_csv(os.path.join(pot_do_podatkov, "obcine.csv"), index_col="ime")

## Nepremičnine

Najprej izračunajmo povprečno vrednosti nepremičnin v posamezni občini. Pri tem se osredotočimo na hiše in stanovanja, ki se prodajajo in si zabeležimo še koliko nepremičnin prištejemo k povprečju.

In [14]:
# Vsem nepremičninam izračunamo ceno kvadratnega metra.
nepremicnine["cena_na_m2"] = nepremicnine.cena / nepremicnine.velikost

# Filtri
hise = nepremicnine.vrsta == "Hiša"
stanovanja = nepremicnine.vrsta == "Stanovanje"
se_prodaja = nepremicnine.posredovanje == "prodaja"

# Zajem
nepremicnine_po_obcinah = nepremicnine[(hise | stanovanja) & se_prodaja].groupby("obcina")
    
# Obdelava
rezultat = nepremicnine_po_obcinah.mean()
rezultat["stevilo_nepremicnin"] = nepremicnine_po_obcinah.size()

rezultat[rezultat.stevilo_nepremicnin > 50].sort_values("cena_na_m2", ascending=False).head(10)

# Priključi podatke k občinam.
obcine["cena_na_m2"] = rezultat.cena_na_m2
obcine["stevilo_nepremicnin"] = rezultat.stevilo_nepremicnin

# obcine[obcine.stevilo_nepremicnin > 50].sort_values("cena_na_m2", ascending=False).head(10)

## Migracije

Izračunajmo sedaj kako se premikajo migracije po Slovenskih regijah.


In [15]:
migracije_po_obcinah = migracije_med_obcinami.groupby('obcina').mean()

urejene_migracije = migracije_po_obcinah.sort_values("delez_migrantov", ascending=False)
urejene_migracije[['delez_migrantov', 'delez_lokalnih_delavcev']]

obcine["delez_migrantov"] = migracije_po_obcinah.delez_migrantov
obcine["delez_lokalnih_delavcev"] = migracije_po_obcinah.delez_lokalnih_delavcev

obcine[obcine.stevilo_nepremicnin > 50].sort_values("cena_na_m2", ascending=False).head(10)

,st_prebivalcev,starost,st_vrtcev,st_otrok_v_vrtcih,delez_otrok_v_vrtcih,st_ucencev_v_osnovni_soli,st_dijakov,st_delovno_aktivnih_po_prebivaliscu,st_delovno_aktivnih_po_delovnem_mestu,placa_bruto,placa_neto,cena_na_m2,stevilo_nepremicnin,delez_migrantov,delez_lokalnih_delavcev
ime,,,,,,,,,,,,,,,
Ljubljana,294113,42.6,128.0,13732.0,84.4,25664,8216,128262,232319,2025.73,1281.66,3065.210486,1866.0,13.965,86.035
Kranjska Gora,5206,48.5,2.0,182.0,87.7,418,144,2124,1559,1553.92,1023.02,2608.584132,55.0,62.460,37.540
Bled,7846,45.6,2.0,296.0,84.1,642,248,3395,3041,1668.38,1083.43,2606.936649,76.0,60.320,39.680
Grosuplje,21265,40.4,17.0,991.0,80.5,2211,824,9895,7294,1569.86,1029.15,2185.629895,81.0,65.375,34.625
Medvode,16651,42.8,8.0,733.0,80.5,1575,619,7302,4529,1691.47,1102.65,1902.166916,90.0,72.355,27.645
Škofljica,11419,40.4,7.0,488.0,85.4,1216,450,5178,2755,1488.12,987.31,1897.659489,72.0,81.865,18.135
Domžale,36429,41.3,17.0,1763.0,84.1,3942,1496,16536,11932,1590.03,1044.06,1859.342924,165.0,64.370,35.630
Kamnik,29847,41.8,19.0,1405.0,85.0,2872,1112,13470,8495,1563.76,1022.91,1654.325380,83.0,57.270,42.730
Kranj,56715,42.6,31.0,2411.0,77.5,5517,1891,25323,25552,1745.10,1132.00,1624.609200,164.0,45.035,54.965


In [16]:
podjetja

,id,ime,maticna_stevilka,obcina,zaposleni,dobicek
0,1519140000,ADECCO H.R. d.o.o.,1519140000,Ljubljana,Nad 1000 zaposlenih,604403.0
1,5479541000,"ADRIA MOBIL, d.o.o. Novo mesto",5479541000,Novo mesto,Nad 1000 zaposlenih,35762836.0
2,1387570000,Akrapovič d.d.,1387570000,Ivančna Gorica,Nad 1000 zaposlenih,16674429.0
3,5518121000,BOXMARK LEATHER d.o.o.,5518121000,Kidričevo,Nad 1000 zaposlenih,-4750308.0
4,5684943000,BSH HIŠNI APARATI d.o.o. Nazarje,5684943000,Nazarje,Nad 1000 zaposlenih,13133119.0
...,...,...,...,...,...,...
9920,5834945000,ŽNIDER'S d.o.o.,5834945000,Maribor,10 do 19 zaposlenih,42135.0
9921,3879534000,ŽUPA d.o.o.,3879534000,Grosuplje,10 do 19 zaposlenih,797.0
9922,2195437000,ŽUPNIJSKI ZAVOD SV. MARTINA KRANJ - STRAŽIŠČE,2195437000,Kranj,10 do 19 zaposlenih,NaN
9923,6121292000,ŽaBa Luka Kovačič s.p.,6121292000,Kranj,10 do 19 zaposlenih,7978.0
